In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import deque

from deap import base
from deap import creator
from deap import tools

In [ ]:


# =============================================
# --- 1. 新しい交叉（Crossover）関数の実装 ---
# =============================================
# (前回と同じコード)
def undx(ind1, ind2, ind3, indpb, bound_l, bound_u):
    size = len(ind1)
    x1, x2, x3 = np.array(ind1), np.array(ind2), np.array(ind3)
    child1, child2 = toolbox.clone(ind1), toolbox.clone(ind2)
    g = (x1 + x2) / 2
    d = x1 - x2
    n = size
    sigma_xi = 0.5
    sigma_eta = 0.35 / np.sqrt(n)
    e1 = np.random.normal(0, sigma_xi) * d
    D = np.linalg.norm(x3 - g)
    e_eta = np.zeros(n)
    for i in range(n):
        v = np.random.normal(0, sigma_eta) * D
        e_eta = e_eta + v
    c1 = g + e1 + e_eta
    c2 = g - e1 - e_eta
    c1 = np.clip(c1, bound_l, bound_u)
    c2 = np.clip(c2, bound_l, bound_u)
    child1[:], child2[:] = c1, c2
    return child1, child2

def spx(parents, bound_l, bound_u):
    n = len(parents[0])
    if len(parents) != n + 1:
        raise ValueError(f"SPX requires n+1 parents, but got {len(parents)} for n={n}")
    x = np.array(parents)
    child = toolbox.clone(parents[0])
    g = np.mean(x, axis=0)
    epsilon = np.random.normal(0, 1, size=n+1)
    c = g + np.sum([eps * (xi - g) for eps, xi in zip(epsilon, x)], axis=0)
    c = np.clip(c, bound_l, bound_u)
    child[:] = c
    return child,

# =======================================
# --- 2. Ackley関数とDEAPの基本設定 ---
# =======================================
# (前回と同じコード)
def evalAckley(individual):
    a, b, c, n = 20, 0.2, 2 * np.pi, len(individual)
    x = np.array(individual)
    sum_sq_term = -b * np.sqrt(np.sum(x**2) / n)
    cos_term = np.sum(np.cos(c * x)) / n
    result = -a * np.exp(sum_sq_term) - np.exp(cos_term) + a + np.e
    return (result,)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()

# ================================
# --- 3. Toolboxセットアップ関数 ---
# ================================
def setup_toolbox(config):
    """CONFIGに基づいてToolboxをセットアップする"""
    toolbox.register("attr_float", random.uniform, config["bound_l"], config["bound_u"])
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=config["ndim"])
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evalAckley)

    # --- 交叉手法の切り替え ---
    if config["crossover_method"] == "SBX":
        toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=config["bound_l"], up=config["bound_u"], eta=20.0)
    elif config["crossover_method"] == "BLX-a":
        toolbox.register("mate", tools.cxBlend, alpha=config["blx_alpha"])
    elif config["crossover_method"] == "UNDX":
        toolbox.register("mate", undx, indpb=1.0, bound_l=config["bound_l"], bound_u=config["bound_u"])
    elif config["crossover_method"] == "SPX":
        toolbox.register("mate", spx, bound_l=config["bound_l"], bound_u=config["bound_u"])
    else:
        raise ValueError(f"Unknown crossover method: {config['crossover_method']}")

    # --- 突然変異 ---
    toolbox.register("mutate", tools.mutPolynomialBounded, low=config["bound_l"], up=config["bound_u"], eta=20.0, indpb=1.0/config["ndim"])
    
    # --- 選択手法の切り替え ---
    if config["selection_method"] == "Tournament":
        toolbox.register("select", tools.selTournament, tournsize=3)
    elif config["selection_method"] == "Roulette":
        # selRouletteは適応度に応じて選択確率が決まる
        # 最小化問題でもDEAPが内部で適切に処理してくれる
        toolbox.register("select", tools.selRoulette)
    else:
        raise ValueError(f"Unknown selection method: {config['selection_method']}")


# =============================
# --- 4. グラフ描画関数 ---
# =============================
# (前回と同じコード)
def plot_log(logbook):
    gen = logbook.select("gen")
    avg = logbook.select("avg")
    std = logbook.select("std")
    min_ = logbook.select("min")
    fig, ax1 = plt.subplots(figsize=(10, 6))
    line1 = ax1.plot(gen, avg, "b-", label="Average Fitness")
    line2 = ax1.plot(gen, min_, "g-.", label="Minimum Fitness")
    ax1.set_xlabel("Generation (or equivalent)")
    ax1.set_ylabel("Fitness")
    std_upper = np.array(avg) + np.array(std)
    std_lower = np.array(avg) - np.array(std)
    ax1.fill_between(gen, std_lower, std_upper, color='b', alpha=0.15, label="Avg +/- Std Dev")
    ax1.legend(loc="upper right")
    plt.title("Fitness over Generations")
    plt.grid(True)
    plt.show()

# =================================
# --- 5. メインの実行部分 ---
# =================================
def main(config):
    random.seed(config["seed"])
    np.random.seed(config["seed"])
    
    setup_toolbox(config)
    
    pop = toolbox.population(n=config["pop_size"])
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    logbook = tools.Logbook()
    logbook.header = "gen", "nevals", "min", "avg", "std"

    # 最初の世代を評価
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    
    hof.update(pop)
    record = stats.compile(pop)
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    print(logbook.stream)

    # --- 世代交代モデルの選択と実行 ---
    n_evals_total = config["ngen"] * config["pop_size"]
    eval_count = len(pop)

    # Case 1: エリート選択付き世代モデル
    if config["generation_model"] == "GenerationalWithElitism":
        for gen in tqdm(range(1, config["ngen"] + 1), desc="GenerationalWithElitism"):
            # 親を選択（エリートの分は引いておく）
            selected_parents = toolbox.select(pop, k=len(pop) - config["n_elites"])
            offspring = list(map(toolbox.clone, selected_parents))
            
            # 交叉と突然変異
            for i in range(1, len(offspring), 2):
                if random.random() < config["cxpb"]:
                    offspring[i-1], offspring[i] = toolbox.mate(offspring[i-1], offspring[i])
                    del offspring[i-1].fitness.values, offspring[i].fitness.values
            for mutant in offspring:
                if random.random() < config["mutpb"]:
                    toolbox.mutate(mutant)
                    del mutant.fitness.values
            
            # エリート個体を追加
            elites = tools.selBest(pop, k=config["n_elites"])
            offspring.extend(elites)
            
            # 評価と更新
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit
            
            pop[:] = offspring
            hof.update(pop)
            record = stats.compile(pop)
            logbook.record(gen=gen, nevals=len(invalid_ind), **record)

    # Case 2: MGG (Minimal Generation Gap)
    elif config["generation_model"] == "MGG":
        with tqdm(total=n_evals_total, desc="MGG", initial=eval_count) as pbar:
            gen_count = 0
            while eval_count < n_evals_total:
                parents = toolbox.select(pop, k=2)
                child1, child2 = toolbox.mate(toolbox.clone(parents[0]), toolbox.clone(parents[1]))
                toolbox.mutate(child1); toolbox.mutate(child2)
                del child1.fitness.values, child2.fitness.values
                
                fitnesses = toolbox.map(toolbox.evaluate, [child1, child2])
                child1.fitness.values, child2.fitness.values = fitnesses[0], fitnesses[1]
                eval_count += 2
                
                worst_indices = sorted(range(len(pop)), key=lambda k: pop[k].fitness.values[0], reverse=True)
                pop[worst_indices[0]], pop[worst_indices[1]] = child1, child2
                
                if eval_count % config["pop_size"] == 0:
                    gen_count += 1
                    hof.update(pop)
                    record = stats.compile(pop)
                    logbook.record(gen=gen_count, nevals=eval_count, **record)
                pbar.update(2)

    # Case 3: JGG (Just Generation Gap)
    elif config["generation_model"] == "JGG":
        if config["crossover_method"] != "UNDX":
            print("Warning: JGG is typically used with multi-parent crossovers like UNDX.")
        
        with tqdm(total=n_evals_total, desc="JGG", initial=eval_count) as pbar:
            gen_count = 0
            while eval_count < n_evals_total:
                # 親を選択: ベスト1個体 + ランダム2個体
                parent1 = hof[0]
                
                # parent1と重複しないように残りの集団から2つ選ぶ
                other_individuals = [ind for ind in pop if ind != parent1]
                p_indices = random.sample(range(len(other_individuals)), 2)
                parent2 = other_individuals[p_indices[0]]
                parent3 = other_individuals[p_indices[1]]

                # 交叉と突然変異で子を生成
                child1, child2 = toolbox.mate(parent1, parent2, parent3)
                toolbox.mutate(child1); toolbox.mutate(child2)
                del child1.fitness.values, child2.fitness.values

                # 評価
                fitnesses = toolbox.map(toolbox.evaluate, [child1, child2])
                child1.fitness.values, child2.fitness.values = fitnesses[0], fitnesses[1]
                eval_count += 2

                # 入れ替え: ランダムに選んだ親2つを子と入れ替える
                # pop.index()で元の集団でのインデックスを探す
                idx2_in_pop = pop.index(parent2)
                idx3_in_pop = pop.index(parent3)
                pop[idx2_in_pop] = child1
                pop[idx3_in_pop] = child2

                # ログ記録
                if eval_count % config["pop_size"] == 0:
                    gen_count += 1
                    hof.update(pop)
                    record = stats.compile(pop)
                    logbook.record(gen=gen_count, nevals=eval_count, **record)
                pbar.update(2)
                
    else:
        raise ValueError(f"Unknown generation model: {config['generation_model']}")
    
    print("\n" + "="*30)
    print("Optimization Finished!")
    print(f"Best individual found: {hof[0]}")
    print(f"Fitness: {hof[0].fitness.values[0]}")
    print("="*30 + "\n")
    
    plot_log(logbook)

if __name__ == "__main__":
    # =================================
    # --- 6. 設定をここに集約！ ---
    # =================================
    CONFIG = {
        # --- 基本設定 ---
        "pop_size": 100,
        "ndim": 30,
        "ngen": 200,
        "seed": 42,
        
        # --- 探索範囲 ---
        "bound_l": -32.768, "bound_u": 32.768,

        # --- アルゴリズム選択 ---
        "generation_model": "JGG",             # "GenerationalWithElitism", "MGG", "JGG"
        "crossover_method": "UNDX",            # "SBX", "BLX-a", "UNDX", "SPX"
        "selection_method": "Tournament",      # "Tournament", "Roulette"
        
        # --- 遺伝的演算子パラメータ ---
        "cxpb": 0.9,
        "mutpb": 0.1,
        
        # --- モデル別パラメータ ---
        "n_elites": 2,
        "blx_alpha": 0.1,
    }

    main(CONFIG)